# Project 4 quick start

***

## Basic usage

This notebook shows a basic usage of the `deepracer_gym` package along with some utility functions in `src.utils`.

Make sure that you have completed the setup from the `SETUP.md` file and are using the proper python environment with this notebook.

### Start the simulation service

Start the simulation service container with the following command. Using this for the very first time may take some time (upto ~10 or 15 minutes).

In [ ]:
%%bash
source scripts/restart_deepracer.sh

You may also find other scripts under `scripts/` similarly useful to stop or restart the simulation service, etc.

To check if the container is rumming you can use the following commands.

In [ ]:
%%bash
apptainer instance list ||  # if using Apptainer (PACE ICE)
docker ps                   # if using Docker (local setup)

Note that the simulator is initialized by the config files in the `configs/` directory. To change the simulation settings, restart it after changing the files in `configs/` accordingly.

### Interact with simulation via `deepracer_gym`

We can interact with the simulation service using the familiar `gymnasium` API via the `deepracer-v0` environment provided by the `deepracer_gym` package (under `packages/`).

Simply import `deepracer_gym` before using the `deepracer-v0` environment with `gymnasium` as usual.

In [ ]:
import gymnasium as gym
import deepracer_gym

env = gym.make(
    'deepracer-v0'
)

observation, info = env.reset()

observation, reward, terminated, truncated, info = env.step(
    env.action_space.sample()
)

env.close()

In [ ]:
# see the output dimensions of the observations
{
    k: v.shape for k, v in observation.items()
}

***

## Utility functions and features

### Flattened environment

Notice that the `observation` variable above is a dictionary (keys are sensor names, values are measurements). Such a data-structure is a bit more difficult to handle than simple vectors, especially for batching purposes. Therefore, we suggest that you use the provided `utils.make_environmrnt` function instead. It 'flattens' the observation space into a single vector space using the `gymnasium.wrappers.FlattenObservation` class.

Additionally, it also wraps the environment with a `gymnasium.wrappers.RecordEpisodeStatistics` class, which can be very convenient for calculating things like episode langth and returns.

Please get familiar with all of these functions/ classes before attempting the project.

In [ ]:
from src.utils import make_environment

env = make_environment(         # just replace gym.make
    'deepracer-v0'
)

observation, info = env.reset()

observation, reward, terminated, truncated, info = env.step(
    env.action_space.sample()
)

env.close()

In [ ]:
# see the output dimensions of the observations
observation.shape

### Visualize agent policy

So long as your agent implements a `get_action` method as in `src.agents.py`, you can use our provided `src.utils.demo` function to visualize the policy of the agent in the form of a MP4 video saved under `demos/`.

In [ ]:
from src.utils import demo
from src.agents import RandomAgent

agent = RandomAgent(environment=env)
demo(agent.eval())

### Evaluate on multiple tracks

You can use the provided `src.utils.evaluate` function to evaluate your agent on all three project tracks (for any specified race type in `configs/environment_configs.yaml`). The results are both returned as well as saved (and over-written) under `evaluations/`.

Please note however that this funciton basically re-starts the simulation a number of times to switch between the tracks for evaluation and this may be a bit time-consuming (~5 minutes on a PACE ICE machine for an untrained agent, and ~20 minutes for a fully trained agent).

In [ ]:
from src.utils import evaluate

metrics = evaluate(
    agent.eval()
)

### Plotting evaluation metrics

You can plot the returned evaluation metrics dictionary using the provided `src.utils.plot_metrics` functions. The results are also saved under `plots/`.

Note however that you do not necessarily have to stick to this exact visualization/plot, and may make adjustments as you see fit.

In [ ]:
from src.utils import plot_metrics

plot_metrics(
    metrics, title='usage'
)

***

## Training and logging

Please use the structure in `src.run.py` to design your training and logging loops. Importantly, try to use `tensorboard` if you can, for example as below.

In [ ]:
from src.run import run

my_hyper_parameters = {
    'lr': 4e-4
}
run(my_hyper_parameters)

To view the training logs, use `tensorboard` by running the following command in your terminal.

```bash
tensorboard --logdir runs
```

***